# Empirical IO PS 2
Maximilian Huber

This code is stored at: https://github.com/MaximilianJHuber/NYU/blob/master/EmpIO/PS2.ipynb.
The notation follows along the lines of Berry, Levinsohn, Pakes (1995): http://www.tcd.ie/Economics/staff/ppwalsh/papers/BLP.pdf

In [1]:
using DataFrames
using GLM
using Optim
using LaTeXStrings

data = readtable("data_ps2.txt", header=false, separator=',')
rename!(data, names(data), [:car, :year, :firm, :price, :quantity, :weight, :hp, :ac, :nest3, :nest4]);

  likely near In[1]:6
  likely near In[1]:6

A car can change characteristics over the years:

In [2]:
data[data[:car] .== 91, :]

, use String instead.
  likely near In[1]:6
  likely near In[1]:6
  likely near In[1]:6
  likely near In[1]:6
  likely near In[1]:6

,car,year,firm,price,quantity,weight,hp,ac,nest3,nest4
1,91,1990,19,5995,52409.0,1620,55,0,1,1
2,91,1991,19,6807,55056.0,1620,55,1,1,1
3,91,1992,19,8219,66046.0,1576,55,1,1,1


 instead.
  likely near In[1]:6


Therefore, one can argue that product and time dimensions collapse into one. But for the time being lets treat good $j$ to be a _car_, time $t$ to be a _year_ and characteristics $x$ to be _weight_, _horse power_ and _air conditioning_ and finally the price $p$ be _price_.

## Part A: Logit

This part follows BLP (1995) section 6.1.

### 1
Agent $i$ derives utility from good $j$ at time $t$ in the following manner:
$$u_{ijt} = \delta_{jt}^* + \epsilon_{ijt} \quad \text{and} \quad \delta_{jt}^* = x'_{jt}\beta_t - \alpha_t p_{jt}+ \xi_{jt} \quad \forall t \in T$$
where $\epsilon_{ijt}$ is an i.i.d. extrem value.
This logit model has market shares:
$$s_{jt} = \frac{e^{\delta_{jt}^*}}{1 + \sum_{k=1}^J e^{\delta_{kt}^*}}$$
Taking logs yields:
$$\log s_{jt} - \log s_{0t} = \delta_{jt}^* = x'_{jt}\beta_t - \alpha_t p_{jt} + \xi_{jt} $$
where $\xi_{jt}$ is the unobservable good and time specific utility.

This equation will be estimated year-by-year.

### 2 
If I had chosen instead to pool the years the derivation would not change, but panel logit relies on the time-independence of $\xi_{jt}$, which is very implausible.

### 3 
Now I estimate the model with GMM using the BLP instrument for $p$:

#### Data Preparation
Market shares are calculated with an assumed market size of 100 million:

In [3]:
data[:share] = data[:quantity] / 1e8;

  likely near In[1]:6
  likely near In[1]:6
  likely near In[1]:6
  likely near In[1]:6

Instruments for the price are constructed by looking at the good's competitors, as defined by the goods produced by other firms, avaiable in the same year. I average of the characteristics of those competing goods.

I normalize weight, horse power and price BEFORE creating the instruments, to get the condition number of the matrices down:

In [4]:
data[:weight] = data[:weight] / mean(data[:weight])
data[:hp]     = data[:hp] / mean(data[:hp])
data[:price]  = data[:price] / mean(data[:price]);

String instead.
  likely near In[1]:6
  likely near In[1]:6

In [5]:
function competitor_mean_characteristic(good, char::Symbol)
    mean(data[(data[:firm] .!= good[:firm]) .* (data[:year] .== good[:year]), char]) #same year, different firm
end

 instead.
  likely near In[1]:6
  likely near In[1]:6

competitor_mean_characteristic (generic function with 1 method)

In [6]:
data[:comp_weight] = [competitor_mean_characteristic(good, :weight) for good in eachrow(data)]
data[:comp_hp]     = [competitor_mean_characteristic(good, :hp) for good in eachrow(data)]
data[:comp_ac]     = [competitor_mean_characteristic(good, :ac) for good in eachrow(data)];

 instead.
  likely near In[1]:6
  likely near In[1]:6
  likely near In[1]:6

And the left hand side of the model is:

In [7]:
data[:LHS] = zeros(size(data)[1])

for y in [1990, 1991, 1992]
    data[data[:year] .== y, :LHS] = 
        log(data[data[:year] .== y, :share]) - log(1 - sum(data[data[:year] .== y, :share]))
end

, use String instead.
  likely near In[1]:6
  likely near In[1]:6
  likely near In[1]:6
  likely near In[1]:6
in #readtable#84 at C:\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\Users\Max\AppData\Local\JuliaPro-0.6.0.1\pkgs-0.6.0.1\v0.6\DataFrames\src\dataframe\io.jl
  likely near In[1]:6
in #readtable#84 at C:\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\Users\Max\AppData\Local\JuliaPro-0.6.0.1\pkgs-0.6.0.1\v0.6\DataFrames\src\dataframe\io.jl

In [8]:
head(data)


  likely near In[1]:6
  likely near In[1]:6

,car,year,firm,price,quantity,weight,hp,ac,nest3,nest4,share,comp_weight,comp_hp,comp_ac,LHS
1,91,1990,19,0.2970233932704021,52409.0,0.5559956020578457,0.4081228050300215,0,1,1,0.00052409,0.9900775851227993,0.9764801886258073,0.4270833333333333,-7.462877776838659
2,35,1990,7,0.4307953969117842,17122.0,0.7619198991163071,0.7420414636909483,0,2,2,0.00017122,1.007985368233883,1.001208568345631,0.45081967213114754,-8.581591922822227
3,61,1990,16,0.43822717489186097,65590.0,0.8974867280131275,0.6900985612325818,0,1,1,0.0006559,1.0072831756766172,1.0122090818191223,0.5043478260869565,-7.238532863836876
4,52,1990,10,0.5360293731096715,49877.0,0.8662548762925941,0.6826781465956724,0,1,1,0.00049877,1.0086561218201344,1.001989689829612,0.47244094488188976,-7.51239613448586
5,26,1990,4,0.6837235741670641,35944.0,0.7488780269692712,0.8607680978814999,0,2,3,0.00035944,1.0075772348148713,0.996691248532191,0.46825396825396826,-7.839993937374661
6,54,1990,11,0.8420204451426995,4640.0,0.9376419659395274,0.9498130735244137,1,2,2,4.64e-5,1.0036015934934468,0.9906541153244582,0.4496124031007752,-9.887241742904356


, use String

#### GMM Estimation
I estimate $\hat{\theta}$ using the following GMM procedure:
$$\max_\theta Q_n\big(\theta \big)$$
where $Q_n\big(\theta\big) = -\frac{1}{2}g_n\big(\theta\big)'\,\hat{W}\,g_n\big(\theta\big)$ and $g_n\big(\theta\big) = \frac{1}{n}\sum_{r=1}^R g\big(w_r; \theta\big)$. 

$g\big(w_r; \theta\big) = z_r \cdot (y_r - x_r \cdot \theta)$ is the residual $\xi_{jt}$ from the model above times the instruments calculated using $w_r$, a row of data.

The efficient GMM is with estimated weighting matrix $\hat{W}_A=\left(\frac{1}{n} \sum_{o=1}^n{g(w_o,\theta_{first})g(w_o,\theta_{first})'}\right)^{-1}$:

In [9]:
# w is data, with the left-hand-side as its first column, z are included regressors and excluded instruments
function gn(w, z, θ)

    1/size(w)[1] .* z' * (w[:,1] - w[:, 2:end] * θ)
    
end

#Wrapper creates a closure around the provided data set 
function Qn_wrapper(w, z)
    return θ -> 1/2 * (gn(w,z,θ)' * gn(w,z,θ))[1,1]
end

function Qn_wrapper(w, z, W)
    return θ -> 1/2 * (gn(w,z,θ)' * W * gn(w,z,θ))[1,1]
end


function g(w, z, θ)
    z * (w[1] - w[2:end]' * θ)
end

function What(w, z, θ)
    result = zeros(Float64, size(z)[2], size(z)[2])
    for i in 1:size(w)[1]
        result .+= g(w[i,:],z[i,:],θ) * g(w[i,:],z[i,:],θ)'
    end
    return inv(result/size(z)[1])
end

 instead.
  likely near In[1]:6
  likely near In[1]:6
  likely near In[1]:6
  likely near In[1]:6


What (generic function with 1 method)

In [10]:
table3 = DataFrame()
table3[:year] = [1990, 1991, 1992]

table3 = hcat(table3, convert(DataFrame, hcat(
    [begin
        w = convert(Array{Float64}, data[data[:year] .== year, [:LHS, :weight, :hp, :ac, :price]])
        z = convert(Array{Float64}, data[data[:year] .== year, [:weight, :hp, :ac, :comp_weight, :comp_hp, :comp_ac]])

        Qn = Qn_wrapper(w, z)

        optres = optimize(Qn, zeros(4), BFGS())
        θfirst = optres.minimizer

        Qn = Qn_wrapper(w, z, What(w, z, θfirst))
        optres = optimize(Qn, θfirst, BFGS())
        optres.minimizer
    end for year in [1990, 1991, 1992]]...)')
)

rename!(table3, names(table3)[2:end], [:βw, :βhp, :βac, :α])
table3[:α] = - table3[:α]

table3

  likely near In[1]:6
in builddf at C:\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\Users\Max\AppData\Local\JuliaPro-0.6.0.1\pkgs-0.6.0.1\v0.6\DataFrames\src\dataframe\io.jl


,year,βw,βhp,βac,α
1,1990,-0.14773648666540135,-19.223330773099537,-3.191339584178792,-13.752715568946247
2,1991,-0.9513776168253341,-15.063324832825115,-1.0394038967574601,-8.6793352740717
3,1992,-1.4523264007228,-13.174051159130832,-0.962987121327918,-6.839267481816083


### 4
Cross-price elasticities are constant over all goods, which is rather unrealistic. For example, increasing the price of a Aston Martin DB11 would shift market share equally to the Ford Prius and the Ferrari 911. 

Similarily, the own-price elasticity is constant, but in reality some cars will have higher and some lower.
## Part B: Nested Logit
### 5
Using Berry (1994) notation:

$$\log s_{jt} - \log s_{0t} = \frac{\delta_{jt}^*}{1-\sigma} - \sigma \log{D_g}$$
where $D_g=\sum_{j \in g} e^{\delta_{jt}^*/(1-\sigma)}$. That yields:

$$\log s_{jt} - \log s_{0t} = x'_{jt}\beta_t - \alpha_t p_{jt} + \sigma \log s_{j|g} + \xi_{jt}  $$
### 6
Let me calculate the within group shares:

In [11]:
for nest_structure in [:nest3, :nest4]
    # initialize a column of zeros
    data[Symbol(string("share_", string(nest_structure)))] = zeros(size(data)[1])

    for y in [1990, 1991, 1992]
        for nest in 1:maximum(data[nest_structure])
            data[(data[:year] .== y) .* (data[nest_structure] .== nest), Symbol(string("share_", string(nest_structure)))] = 
                log(data[(data[:year] .== y) .* (data[nest_structure] .== nest), :share]) - 
                log(sum(data[(data[:year] .== y) .* (data[nest_structure] .== nest), :share]))
        end
    end
end

And the within group competitor's characteristics, which are being used as an instrument for the within shares:

In [12]:
function nest_competitor_mean_characteristic(good, char::Symbol, nest_structure::Symbol)
    mean(data[(data[nest_structure] .== good[nest_structure]) .* (data[:year] .== good[:year]) .* 
            (data[:car] .!= good[:car]), char]) #same year, same nest, different car
end

nest_competitor_mean_characteristic (generic function with 1 method)

In [13]:
for nest_structure in [:nest3, :nest4]

    data[Symbol(string("comp_weight_", string(nest_structure)))] = 
        [nest_competitor_mean_characteristic(good, :weight, nest_structure) for good in eachrow(data)]

    data[Symbol(string("comp_hp_", string(nest_structure)))] = 
        [nest_competitor_mean_characteristic(good, :hp, nest_structure) for good in eachrow(data)]

    data[Symbol(string("comp_ac_", string(nest_structure)))] = 
        [nest_competitor_mean_characteristic(good, :ac, nest_structure) for good in eachrow(data)];
    
end

In [14]:
head(data)

,car,year,firm,price,quantity,weight,hp,ac,nest3,nest4,share,comp_weight,comp_hp,comp_ac,LHS,share_nest3,share_nest4,comp_weight_nest3,comp_hp_nest3,comp_ac_nest3,comp_weight_nest4,comp_hp_nest4,comp_ac_nest4
1,91,1990,19,0.2970233932704021,52409.0,0.5559956020578457,0.4081228050300215,0,1,1,0.00052409,0.9900775851227993,0.9764801886258073,0.4270833333333333,-7.462877776838659,-4.304576788942753,-3.684711188040598,0.9528997625875185,0.7518815787529369,0.2608695652173913,0.9115978224718131,0.6863883539141272,0.11538461538461539
2,35,1990,7,0.4307953969117842,17122.0,0.7619198991163071,0.7420414636909483,0,2,2,0.00017122,1.007985368233883,1.001208568345631,0.45081967213114754,-8.581591922822227,-5.573016845671248,-5.7614217762818,1.009372262748225,1.027351029368062,0.4927536231884058,0.9905522864668314,0.927348530582537,0.4246575342465753
3,61,1990,16,0.43822717489186097,65590.0,0.8974867280131275,0.6900985612325818,0,1,1,0.0006559,1.0072831756766172,1.0122090818191223,0.5043478260869565,-7.238532863836876,-4.08023187594097,-3.460366275038815,0.9454760424580558,0.7457516710094029,0.2608695652173913,0.89846354839661,0.6755431325217209,0.11538461538461539
4,52,1990,10,0.5360293731096715,49877.0,0.8662548762925941,0.6826781465956724,0,1,1,0.00049877,1.0086561218201344,1.001989689829612,0.47244094488188976,-7.51239613448586,-4.354095146589954,-3.7342295456877985,0.9461549957563283,0.7459129843710749,0.2608695652173913,0.8996647734627844,0.675828533084679,0.11538461538461539
5,26,1990,4,0.6837235741670641,35944.0,0.7488780269692712,0.8607680978814999,0,2,3,0.00035944,1.0075772348148713,0.996691248532191,0.46825396825396826,-7.839993937374661,-4.831418860223682,-3.3802007968344165,1.0095612753880372,1.025630353510228,0.4927536231884058,1.1384493562225968,1.3454560903014512,0.8636363636363636
6,54,1990,11,0.8420204451426995,4640.0,0.9376419659395274,0.9498130735244137,1,2,2,4.64e-5,1.0036015934934468,0.9906541153244582,0.4496124031007752,-9.887241742904356,-6.878666665753377,-7.067071596363929,1.0068255661275984,1.0243398466168523,0.4782608695652174,0.9881451348665133,0.924502344146462,0.410958904109589


#### 6.a.

In [15]:
nest_structure = :nest3

table6 = DataFrame()
table6[:year] = [1990, 1991, 1992, 1990, 1991, 1992]
table6[:nesting] = [3, 3, 3, 4, 4, 4]

table6 = hcat(table6, convert(DataFrame, hcat(
    [begin
        # do 2SLS:
        X = convert(Array{Float64}, data[data[:year] .== year, 
                [:weight, :hp, :ac, :price, Symbol(string("share_", string(nest_structure)))]])
        Z = convert(Array{Float64}, data[data[:year] .== year, 
                [:weight, :hp, :ac, :comp_weight, :comp_hp, :comp_ac, Symbol(string("comp_weight_", string(nest_structure))), 
                Symbol(string("comp_hp_", string(nest_structure))), Symbol(string("comp_ac_", string(nest_structure)))]])

        y = convert(Array{Float64}, data[data[:year] .== year, [:LHS]])

        θ2SLS = ((X'Z * (Z'Z)^(-1) * Z'X) \ (X'Z * (Z'Z)^(-1) * Z'y))[:,1]

        # so efficient GMM starting from 2SLS:
        w = hcat(data[data[:year] .== year, :LHS], X)
        z = Z

        Qn = Qn_wrapper(w, z)

        optres = optimize(Qn, θ2SLS, BFGS())
        θfirst = optres.minimizer

        Qn = Qn_wrapper(w, z, What(w, z, θfirst))
        optres = optimize(Qn, θfirst, BFGS())
        optres.minimizer
    end for nest_structure in [:nest3, :nest4] for year in [1990, 1991, 1992]] ...)')
)

rename!(table6, names(table6)[2:end], [:nest, :βw, :βhp, :βac, :α, :σ])
table6[:α] = - table6[:α]

table6

,year,nest,βw,βhp,βac,α,σ
1,1990,3,-1.2719376209090105,-4.379978695470563,0.05076184825827891,-2.2999502733401953,0.9690951928539002
2,1991,3,-0.8301643487342931,-2.0238276128900345,-0.003952554361051232,-0.8623334389041235,1.2558420455246175
3,1992,3,-1.473254567620196,-2.534228169868762,-0.1850286135541211,-1.209403792682061,1.0664739428544119
4,1990,4,-0.14163499647361066,-7.549133203378204,-0.15951471552449165,-3.851278212355502,0.8588797261800747
5,1991,4,-1.2460353568693108,-5.9523095223955655,0.19833234933197522,-2.872992249777215,0.7866821695497277
6,1992,4,2.0306936842297287,-7.2526842155543525,-0.44886072785761605,-2.3442116911240403,1.1040830493657352


The estimates for the price coefficients are higher!

#### 6.b.
The conditional error must be time-independent. This is very implausible with real data, since brands have autocorrelation in how much they are liked by buyers.

#### 6.c.
$\sigma$ is often close to one, but in 1991 the two nesting structures give very different estimates for $\sigma$. With 4 nests the $\sigma=0.79$ indicates there is much correlation within the nest. With 3 nests the $\sigma=1.26$ might not be consistent with random utility maximization! 
#### 6.d.
The $\alpha$ also varies a lot over the three years! 
### 7
#### 7.a.
Lets have a look at price elasticities in the logit and the nested logit model:

|          |   Standard Logit   | Nested Logit  |
|:-:|:-:|:-:|
| $\frac{\partial s_{jt}}{\partial p_{jt}}$  | $-\alpha s_{jt}\big(1-s_{jt}\big)$  | $-\alpha s_{jt}\big(\frac{1}{1-\sigma}-\frac{\sigma}{1-\sigma}s_{jt|g}-s_{jt}\big)$  |
|  $\frac{\partial s_{jt}}{\partial p_{kt}}$ | $\alpha s_{jt}s_{kt}$  | $\quad\quad\begin{cases}
\alpha s_{kt}\big(\frac{\sigma}{1-\sigma}s_{jt|g}+s_{jt}\big) & \text{same}\\
\alpha s_{jt}s_{kt} & \text{different}
\end{cases}$  |

The nested logit allows for a more flexible substitution pattern!
#### 7.b.
One could grow a dicision tree (or use some other machine learning methods, "nearest neighbor", etc.) to flexibly use the existing data (characteristics only, no market shares) to create nests. As validation I would check whether the catagories resemble something like: hypercar/supercar, sports car, SUV, hatchback, ...
#### 7.c.
We need to define the nesting structure a priori and must not make any mistakes. It is clear that we would need multiple layers of nesting to get a realistic classification, but then it is hard to justify why to split first according to one criterium instead of another.

### 8
#### 8.a.
Probit would allow for a very flexible substitution pattern, but with more than 120 cars per year the covariance matrix would get very big!
#### 8.b.
The pure characteristics model could not reflect the (unobserved) emotional value of people buying an Alfa Romeo Giulia Quadrifoglio although it is a bad car for the price.

## Part C: Mixed Logit
### 9
Now:
$$u_{ijt} = x'_{jt}\beta_t - \alpha_{i} p_{jt}+ \xi_{jt} + \epsilon_{ijt} \quad \forall t \in T$$
where $\alpha_i=\frac{1}{y_i}$ and $y_i \sim \mathcal{LN}$, the log-normal distribution.

Compared to BLP (1995) this is different in the sense the my $\delta$ is now $x'_{jt}\beta_t - \mu_\alpha p_{jt} + \xi_{jt}$, the $\alpha$ is integrated out and its distributional parameters are estimated.

The mixed logit shares are in this setting:
$$s_{jt}=\int_{0}^{\infty}\frac{exp\big(x'_{jt}\beta_{t}-(\mu_{\alpha}-\mu_{\alpha}+\frac{1}{y_{i}})p_{jt}+\xi_{jt}\big)}{1+\sum_{k}exp\big(x'_{jt}\beta_{t}-\frac{1}{y_{i}}p_{jt}+\xi_{jt}\big)}\frac{1}{y_{i}\sigma_{y}\sqrt{2\pi}}\exp\Big(-\frac{(\log y_{i}-\mu_{y})^{2}}{2\sigma_{y}^{2}}\Big)dy_{i}=
$$

$$\int_{0}^{\infty}\frac{exp\big(\delta_{jt}+(\mu_{\alpha}-\frac{1}{y_{i}})p_{jt}\big)}{1+\sum_{k}exp\big(\delta_{jt}+(\mu_{\alpha}-\frac{1}{y_{i}})p_{jt}\big)}\frac{1}{y_{i}\sigma_{y}\sqrt{2\pi}}\exp\Big(-\frac{(\log y_{i}-\mu_{y})^{2}}{2\sigma_{y}^{2}}\Big)dy_{i}=
$$

$$\int_{0}^{\infty}\frac{exp\big(\delta_{jt}+(\mu_{\alpha}-\frac{1}{\exp(e_{i})})p_{jt}\big)}{1+\sum_{k}exp\big(\delta_{kt}+(\mu_{\alpha}-\frac{1}{\exp(e_{i})})p_{kt}\big)}\frac{1}{\exp(e_{i})\sigma_{y}\sqrt{2\pi}}\exp\Big(-\frac{(e_{i}-\mu_{y})^{2}}{2\sigma_{y}^{2}}\Big)\exp(e_{i})de_{i}=
$$

$$\int_{0}^{\infty}\frac{exp\Big(\delta_{jt}+(\mu_{\alpha}-\exp(-e_{i}))p_{jt}\Big)}{1+\sum_{k}exp\Big(\delta_{kt}+(\mu_{\alpha}-\exp(-e_{i}))p_{kt}\Big)}\frac{1}{\sigma_{y}\sqrt{2\pi}}\exp\Big(-\frac{(e_{i}-\mu_{y})^{2}}{2\sigma_{y}^{2}}\Big)de_{i}=
$$

$$\int_{0}^{\infty}\underset{h(e_{i};\delta_{jt},\,p_{jt})}{\underbrace{\frac{1}{\sqrt{\pi}}\frac{exp\Big(\delta_{jt}+\Big(\exp(-\mu_{y}+\frac{1}{2}\sigma_{y}^{2})-\exp(-\mu_{y}-\sqrt{2}\sigma_{y}e_{i})\Big)\,p_{jt}\Big)}{1+\sum_{k}exp\Big(\delta_{kt}+\Big(\exp(-\mu_{y}+\frac{1}{2}\sigma_{y}^{2})-\exp(-\mu_{y}-\sqrt{2}\sigma_{y}e_{i})\Big)\,p_{kt}\Big)}}}\phi(e_{i})de_{i}\approx
$$

$$\approx\sum_{l=1}^n q_l \, h\Big(e_{l};\delta_{jt},\,p_{jt}\Big)$$
where $q_l$ and $e_l$ are the Gauss-Hermite weights and points.

In [21]:
using FastGaussQuadrature

function gn_BLP(w, z, θ)
    1/size(w)[1] .* z' * (w[:,1] .- w[:, 2:4] * θ[1:3] .+ θ[4] .* w[:, 5])
end

#Wrapper creates a closure around the provided data set 
function Qn_wrapper_BLP(w, z)
    
    δ = zeros(size(w)[1])
    (e, q) = gausshermite(30)
    p = w[:, 5] #price
    s = w[:, 1] #share
    
    
    return θ -> begin #w, hp, ac, mu, sigma
        μ_y = θ[4]
        σ_y = θ[5]
        
        h(e) = 1/(√π) .* exp.(δ .+ (exp(-μ_y + 1/2 * σ_y^2) - exp(-μ_y - (√2) * σ_y * e)) .* p) / 
            (1 + sum(exp.(δ .+ (exp(-μ_y + 1/2 * σ_y^2) - exp(-μ_y - (√2) * σ_y * e)) .* p)))
        
        error = 1
        
        while error > 1e-13
            δold = copy(δ)
            δ = δold .+ log.(s) .- log.(hcat(h.(e)...) * q)
            error = maximum(abs.(δ - δold))
        end

        gvec = gn_BLP(hcat(δ, w[:, 2:5]), z, θ) 
        1/2 * (gvec' * gvec)[1,1]
    end
end

Qn_wrapper_BLP (generic function with 1 method)

In [26]:
year = 1990

w = convert(Array{Float64}, data[data[:year] .== year, [:share, :weight, :hp, :ac, :price]])
z = convert(Array{Float64}, data[data[:year] .== year, [:weight, :hp, :ac, :comp_weight, :comp_hp, :comp_ac]])

Qn = Qn_wrapper_BLP(w, z)

optres = optimize(Qn, vcat(zeros(4), 1), NelderMead())

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [0.0,0.0,0.0,0.0,1.0]
 * Minimizer: [-5.691735033022672,-5.0549542860660175, ...]
 * Minimum: 1.126676e-02
 * Iterations: 1000
 * Convergence: false
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: false
   * Reached Maximum Number of Iterations: true
 * Objective Function Calls: 1403

In [27]:
optres = optimize(Qn, optres.minimizer, NelderMead())

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [-5.691735033022672,-5.0549542860660175, ...]
 * Minimizer: [-5.691735033022672,-5.0549542860660175, ...]
 * Minimum: 1.126676e-02
 * Iterations: 1000
 * Convergence: false
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: false
   * Reached Maximum Number of Iterations: true
 * Objective Function Calls: 1007

In [28]:
optres = optimize(Qn, optres.minimizer, NelderMead())

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [-5.691735033022672,-5.0549542860660175, ...]
 * Minimizer: [-5.691735033022672,-5.0549542860660175, ...]
 * Minimum: NaN
 * Iterations: 1000
 * Convergence: false
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: false
   * Reached Maximum Number of Iterations: true
 * Objective Function Calls: 1007

In [29]:
optres = optimize(Qn, optres.minimizer, NelderMead())

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [-5.691735033022672,-5.0549542860660175, ...]
 * Minimizer: [-5.691735033022672,-5.0549542860660175, ...]
 * Minimum: NaN
 * Iterations: 1000
 * Convergence: false
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: false
   * Reached Maximum Number of Iterations: true
 * Objective Function Calls: 1007

In [ ]:
optres = optimize(Qn, optres.minimizer, NelderMead(), Optim.OptimizationOptions(iterations=100000))

Stacktrace:
 [1] depwarn(::